In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from scipy.stats import zscore

##### Merge Files ######################################################################################################################

# Paths to group score CSV files
STEP_1_OUT_PATH = '/content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/output/Step 1 Out'
OUTPUT_PATH = '/content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output'

# Load group scores
mortality_scores = pd.read_csv(f"{STEP_1_OUT_PATH}/Outcome_mortality.csv").rename(columns={"grp_score": "Std_Outcomes_Mortality_score"})
readmission_scores = pd.read_csv(f"{STEP_1_OUT_PATH}/Outcome_readmission.csv").rename(columns={"grp_score": "Std_Outcomes_Readmission_score"})
safety_scores = pd.read_csv(f"{STEP_1_OUT_PATH}/Outcome_safety.csv").rename(columns={"grp_score": "Std_Outcomes_Safety_score"})
pt_exp_scores = pd.read_csv(f"{STEP_1_OUT_PATH}/PtExp.csv").rename(columns={"grp_score": "Std_PatientExp_score"})
process_scores = pd.read_csv(f"{STEP_1_OUT_PATH}/Process.csv").rename(columns={"grp_score": "Std_Process_score"})

# Merge all group scores into one DataFrame
all_scores = mortality_scores.merge(
    readmission_scores[['PROVIDER_ID', 'Std_Outcomes_Readmission_score']],
    on='PROVIDER_ID', how='outer'
).merge(
    safety_scores[['PROVIDER_ID', 'Std_Outcomes_Safety_score']],
    on='PROVIDER_ID', how='outer'
).merge(
    pt_exp_scores[['PROVIDER_ID', 'Std_PatientExp_score']],
    on='PROVIDER_ID', how='outer'
).merge(
    process_scores[['PROVIDER_ID', 'Std_Process_score']],
    on='PROVIDER_ID', how='outer'
)

# Save the merged data to the output directory
merged_output_path = f"{OUTPUT_PATH}/All_Group_Scores.csv"
all_scores.to_csv(merged_output_path, index=False)

print(f"Group scores merged successfully. File saved at: {merged_output_path}")

##### Summary Score ######################################################################################################################

# Load the merged data
merged_file_path = f"{OUTPUT_PATH}/All_Group_Scores.csv"
data = pd.read_csv(merged_file_path)

# Fixed standard weights with redistributed patient experience weight
weights = {
    "Std_PatientExp_score": 0.11,  # Halved weight for patient experience
    "Std_Outcomes_Readmission_score": 0.245,  # Redistributed weights
    "Std_Outcomes_Mortality_score": 0.245,
    "Std_Outcomes_Safety_score": 0.245,
    "Std_Process_score": 0.155
}

# Add weights to DataFrame
for group, weight in weights.items():
    data[f"std_weight_{group}"] = weight

# Indicator for missing group scores
for group in weights.keys():
    data[f"I_{group}"] = data[group].isna().astype(int)

# Redistribute weights
for group in weights.keys():
    redistribution_factor = 1 - sum(data[f"I_{col}"] * weights[col] for col in weights.keys())
    data[f"weight_{group}"] = np.where(
        data[f"I_{group}"] == 1,
        np.nan,  # Keep missing if group is missing
        data[f"std_weight_{group}"] / redistribution_factor
    )

# Calculate weighted scores
for group in weights.keys():
    data[f"sum_weight_ave_{group}"] = data[f"weight_{group}"] * data[group]

# Calculate the final summary score
weighted_cols = [f"sum_weight_ave_{group}" for group in weights.keys()]
data["summary_score"] = data[weighted_cols].sum(axis=1)

# Save the resulting summary scores to a new CSV file
summary_output_path = f"{OUTPUT_PATH}/Summary_Scores.csv"
data.to_csv(summary_output_path, index=False)

print(f"Summary scores calculated successfully. File saved at: {summary_output_path}")

##### Reporting >3 Measures #############################################################################################################

MEASURE_ANALYSIS = "/content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/output/Step 0 Out/Std_data_2024Jul_analysis.csv"
REPORT_OUTPUT_PATH = f"{OUTPUT_PATH}/Report_Indicator.csv"

# Load the standardized data
data = pd.read_csv(MEASURE_ANALYSIS)

# Define groups with correct column names
measure_OM = ['std_MORT_30_AMI', 'std_MORT_30_CABG', 'std_MORT_30_COPD', 'std_MORT_30_HF',
              'std_MORT_30_PN', 'std_MORT_30_STK', 'std_PSI_4_SURG_COMP']
measure_OS = ['std_COMP_HIP_KNEE', 'std_HAI_1', 'std_HAI_2', 'std_HAI_3', 'std_HAI_4',
              'std_HAI_5', 'std_HAI_6', 'std_PSI_90_SAFETY']
measure_OR = ['std_EDAC_30_AMI', 'std_EDAC_30_HF', 'std_EDAC_30_PN', 'std_OP_32',
              'std_READM_30_CABG', 'std_READM_30_COPD', 'std_READM_30_HIP_KNEE',
              'std_READM_30_HOSP_WIDE', 'std_OP_35_ADM', 'std_OP_35_ED', 'std_OP_36']
measure_PtExp = ['std_H_COMP_1_STAR_RATING', 'std_H_COMP_2_STAR_RATING', 'std_H_COMP_3_STAR_RATING',
                 'std_H_COMP_5_STAR_RATING', 'std_H_COMP_6_STAR_RATING', 'std_H_COMP_7_STAR_RATING',
                 'std_H_GLOB_STAR_RATING', 'std_H_INDI_STAR_RATING']
measure_Process = ['std_HCP_COVID_19', 'std_IMM_3', 'std_OP_10', 'std_OP_13', 'std_OP_18B',
                   'std_OP_22', 'std_OP_23', 'std_OP_29', 'std_OP_3B', 'std_OP_8', 'std_PC_01', 'std_SEP_1']

# Count measures for each group
data['Outcomes_Mortality_cnt'] = data[measure_OM].notnull().sum(axis=1)
data['Outcomes_Safety_cnt'] = data[measure_OS].notnull().sum(axis=1)
data['Outcomes_Readmission_cnt'] = data[measure_OR].notnull().sum(axis=1)
data['Patient_Experience_cnt'] = data[measure_PtExp].notnull().sum(axis=1)
data['Process_cnt'] = data[measure_Process].notnull().sum(axis=1)

# Calculate Total Measure Group Count
data['Total_measure_group_cnt'] = (
    (data['Outcomes_Mortality_cnt'] >= 3).astype(int) +
    (data['Outcomes_Safety_cnt'] >= 3).astype(int) +
    (data['Outcomes_Readmission_cnt'] >= 3).astype(int) +
    (data['Patient_Experience_cnt'] >= 3).astype(int) +
    (data['Process_cnt'] >= 3).astype(int)
)

# Calculate MortSafe Group Count
data['MortSafe_Group_cnt'] = (
    (data['Outcomes_Mortality_cnt'] >= 3).astype(int) +
    (data['Outcomes_Safety_cnt'] >= 3).astype(int)
)

# Determine Reporting Indicator
data['report_indicator'] = (
    (data['MortSafe_Group_cnt'] >= 1) & (data['Total_measure_group_cnt'] >= 3)
).astype(int)

# Assign Peer Grouping
def assign_peer_group(row):
    if row['Total_measure_group_cnt'] == 3:
        return '1) # of groups=3'
    elif row['Total_measure_group_cnt'] == 4:
        return '2) # of groups=4'
    elif row['Total_measure_group_cnt'] == 5:
        return '3) # of groups=5'
    return None

data['cnt_grp'] = data.apply(assign_peer_group, axis=1)

# Save the Report Indicator Output
data[['PROVIDER_ID', 'report_indicator', 'Patient_Experience_cnt', 'Outcomes_Readmission_cnt',
      'Outcomes_Mortality_cnt', 'Outcomes_Safety_cnt', 'Process_cnt',
      'Total_measure_group_cnt', 'MortSafe_Group_cnt', 'cnt_grp']].to_csv(REPORT_OUTPUT_PATH, index=False)

print(f"Report indicator and grouping saved to {REPORT_OUTPUT_PATH}")

##### K-means Clustering ##############################################################################################################

# Load the report macro results
data = pd.read_csv(REPORT_OUTPUT_PATH)
summary_scores = pd.read_csv(summary_output_path)

# Merge `summary_score` into `data`
if 'summary_score' not in data.columns:
    data = data.merge(summary_scores[['PROVIDER_ID', 'summary_score']], on='PROVIDER_ID', how='left')

# Function to perform K-means clustering
def kmeans_clustering(data, peer_group, n_clusters=5):
    group_data = data[(data['cnt_grp'] == peer_group) & (data['report_indicator'] == 1)].copy()
    if group_data.empty:
        return pd.DataFrame()
    quintiles = np.percentile(group_data['summary_score'], [20, 40, 60, 80])
    quintile_medians = [
        group_data[group_data['summary_score'] <= quintiles[0]]['summary_score'].median(),
        group_data[(group_data['summary_score'] > quintiles[0]) & (group_data['summary_score'] <= quintiles[1])]['summary_score'].median(),
        group_data[(group_data['summary_score'] > quintiles[1]) & (group_data['summary_score'] <= quintiles[2])]['summary_score'].median(),
        group_data[(group_data['summary_score'] > quintiles[2]) & (group_data['summary_score'] <= quintiles[3])]['summary_score'].median(),
        group_data[group_data['summary_score'] > quintiles[3]]['summary_score'].median()
    ]
    initial_centers = np.array([m for m in quintile_medians if not np.isnan(m)]).reshape(-1, 1)
    kmeans_phase1 = KMeans(n_clusters=n_clusters, init=initial_centers, n_init=1, max_iter=1000, random_state=42)
    group_data['cluster'] = kmeans_phase1.fit_predict(group_data[['summary_score']])
    group_data = group_data[np.abs(zscore(group_data['summary_score'])) < 3]
    kmeans_phase2 = KMeans(n_clusters=n_clusters, init=kmeans_phase1.cluster_centers_, n_init=1, max_iter=1000, random_state=42)
    group_data['cluster'] = kmeans_phase2.fit_predict(group_data[['summary_score']])
    cluster_means = group_data.groupby('cluster')['summary_score'].mean().sort_values()
    cluster_to_star = {cluster: star for star, cluster in enumerate(cluster_means.index, start=1)}
    group_data['star'] = group_data['cluster'].map(cluster_to_star)
    return group_data[['PROVIDER_ID', 'star', 'cnt_grp']]

# Apply K-means to each peer group
peer_groups = data['cnt_grp'].unique()
all_clusters = []
for peer_group in peer_groups:
    clustered_data = kmeans_clustering(data, peer_group)
    if not clustered_data.empty:
        all_clusters.append(clustered_data)

# Combine results for all peer groups
all_clusters_df = pd.concat(all_clusters, ignore_index=True)
data = data.merge(all_clusters_df, on=['PROVIDER_ID', 'cnt_grp'], how='left')
data['star'] = np.where(data['report_indicator'] == 0, "N/A", data['star'])

# Save final results
final_output_path = f"{OUTPUT_PATH}/Star_Ratings.csv"
data.to_csv(final_output_path, index=False)
print(f"K-means clustering and star ratings completed. Results saved to {final_output_path}.")


Group scores merged successfully. File saved at: /content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output/All_Group_Scores.csv
Summary scores calculated successfully. File saved at: /content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output/Summary_Scores.csv
Report indicator and grouping saved to /content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output/Report_Indicator.csv
K-means clustering and star ratings completed. Results saved to /content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output/Star_Ratings.csv.


# Filter for riverside

In [4]:
import pandas as pd

# Path to the Star Ratings file
STAR_RATINGS_FILE = "/content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output/Star_Ratings.csv"

# List of Riverside CCNs (ensure these are strings)
riverside_ccns = ["490037", "490052", "490130", "490143"]

# Read the star ratings file
star_ratings = pd.read_csv(STAR_RATINGS_FILE)

# Ensure PROVIDER_ID is treated as a string
star_ratings['PROVIDER_ID'] = star_ratings['PROVIDER_ID'].astype(str)

# Filter for the Riverside hospitals
riverside_star_ratings = star_ratings[star_ratings['PROVIDER_ID'].isin(riverside_ccns)]

# Display the filtered results
if not riverside_star_ratings.empty:
    print("Star Ratings for Riverside Hospitals:")
    print(riverside_star_ratings)
else:
    print("No matching CCNs found for Riverside hospitals in the dataset.")

# Save the filtered results to a new CSV file
output_path = "/content/drive/My Drive/Capstone Project/SAS Star Data/Source Code/Analysis/PtExp Scenario Analysis Output/Riverside_Star_Ratings.csv"
riverside_star_ratings.to_csv(output_path, index=False)

print(f"Filtered star ratings saved to: {output_path}")


Star Ratings for Riverside Hospitals:
     PROVIDER_ID  report_indicator  Patient_Experience_cnt  \
4199      490037                 1                       8   
4210      490052                 1                       8   
4249      490130                 1                       8   
4253      490143                 1                       8   

      Outcomes_Readmission_cnt  Outcomes_Mortality_cnt  Outcomes_Safety_cnt  \
4199                         8                       5                    2   
4210                        10                       7                    7   
4249                         7                       5                    3   
4253                         7                       5                    3   

      Process_cnt  Total_measure_group_cnt  MortSafe_Group_cnt  \
4199            9                        4                   1   
4210           11                        5                   2   
4249            9                        5               